In [ ]:
!pip install -U pathway bokeh

In [ ]:
from google.colab import files

# This opens file picker to upload your CSV dataset
uploaded = files.upload()

In [ ]:
import pandas as pd

# Automatically get uploaded filename
filename = list(uploaded.keys())[0]
df_stream = pd.read_csv(filename)
df_stream.head()

In [ ]:
import pathway as pw
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Constants
alpha, beta, gamma, delta, epsilon = 1.2, 0.8, 0.5, 1.0, 1.0
lambda_factor = 0.8
base_price = 10
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}

# Mock coordinates for 14 lots
lot_locations = {
    f"Lot {i}": (12.95 + i * 0.001, 77.6 + i * 0.001) for i in range(1, 15)
}

def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371
    return c * r

In [ ]:
class ParkingInput(pw.Schema):
    LotID: str
    Occupancy: int
    Capacity: int
    Queue: int
    Traffic: float
    SpecialDay: int
    VehicleType: str
    Timestamp: int

@pw.udf
def compute_price_with_competition(lot_id, occupancy, capacity, queue, traffic, special_day, vehicle_type):
    weight = vehicle_weights.get(vehicle_type, 1.0)
    demand = alpha * (occupancy / capacity) + beta * queue - gamma * traffic + delta * special_day + epsilon * weight
    norm_demand = max(0, min(1, (demand - 1.5) / (10 - 1.5)))
    base = base_price * (1 + lambda_factor * norm_demand)

    lat1, lon1 = lot_locations.get(lot_id, (0, 0))
    nearby_prices = []

    for other_lot, (lat2, lon2) in lot_locations.items():
        if other_lot == lot_id:
            continue
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist <= 1.0:
            simulated_price = base_price * (1 + 0.05 * np.random.randint(0, 5))
            nearby_prices.append(simulated_price)

    avg_competitor = np.mean(nearby_prices) if nearby_prices else base_price

    if occupancy >= capacity and base > avg_competitor:
        base = avg_competitor - 1

    return round(max(5, min(base, 20)), 2)

In [ ]:
t = pw.io.csv.read(filename, schema=ParkingInput, mode="streaming")

out = t.select(
    LotID=t.LotID,
    Price=compute_price_with_competition(
        t.LotID, t.Occupancy, t.Capacity, t.Queue, t.Traffic, t.SpecialDay, t.VehicleType
    ),
    Timestamp=t.Timestamp
)

pw.io.jsonlines.write(out, "out.jsonl")
pw.run()

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
import json

output_notebook()

records = []
with open("out.jsonl", "r") as f:
    for line in f:
        records.append(json.loads(line))

df_out = pd.DataFrame(records)
df_out = df_out.sort_values(by="Timestamp")

selected_lot = "Lot 1"
lot_data = df_out[df_out["LotID"] == selected_lot]

source = ColumnDataSource(lot_data)

p = figure(title=f"Dynamic Pricing - {selected_lot}",
           x_axis_label="Timestamp", y_axis_label="Price ($)",
           plot_height=350, plot_width=700)

p.line(x='Timestamp', y='Price', source=source, line_width=2, color='navy')
p.circle(x='Timestamp', y='Price', source=source, size=6, color='red')

show(p)